<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Multi-Variable Logistic Regression and Classification Matrix

_Authors: Sam Stack(DC)_


**Exercise Objectives**
- Hand on experience using Multi-Variable Logistic Regression
- Review and Exploration of the Classification Matrix and its evaluation Metrics
- Introduction to One vs. One and One vs. Rest Classifiers.

**Lets get some data.**
One of the most popular classification datasets for Machine learning is the Iris Dataset, which can be loaded directly from `sklearn.datasets`
- Sklearn datasets are imported as dictionaries and use keys to access specific aspects.
    - `iris.data` : actual matrix of observations
    - `iris.target` : target column for classification
    - `iris.feature_names` :  column names

In [3]:
import seaborn as sns
import pandas as pd
from sklearn import datasets

iris = datasets.load_iris()

In [22]:
# Examine the data
def clean_col_name(feature_name):
    col_name = feature_name.replace(' ', '_').replace('(','').replace(')','')
    return col_name

col_names =  [clean_col_name(col) for col in iris.feature_names]

X = pd.DataFrame(iris.data, columns = col_names)
y = iris.target



In [26]:
pd.Series(y).value_counts()

2    50
1    50
0    50
dtype: int64

**Break down of classes**  
0 : Setosa  
1 : Versicolour  
2 : Virginica  

----

**Modelling**
This data is extreamly neat and tidy so no cleaning necessary and we can get right into modelling.

In [38]:
# model the data, use a cross validation technique as well
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

log_mod = LogisticRegression()
knn_mod = KNeighborsClassifier()
knn_mod.fit(X_train, y_train)
log_mod.fit(X_train,y_train)
y_preds = log_mod.predict(X_test)
y_probs = log_mod.predict_proba(X_test)
y_preds_knn = knn_mod.predict(X_test)
y_probs_knn = knn_mod.predict(X_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [37]:
# model.predict
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score

confusion_matrix(y_test, y_preds)

array([[10,  0,  0],
       [ 0,  9,  0],
       [ 0,  0, 11]])

In [39]:
# evaluated model preformance with a confusion matrix.
confusion_matrix(y_test, y_preds_knn)

array([[10,  0,  0],
       [ 0,  9,  0],
       [ 0,  0, 11]])

In [41]:
log_mod.coef_, X.columns

(array([[ 0.3711229 ,  1.409712  , -2.15210117, -0.95474179],
        [ 0.49400451, -1.58897112,  0.43717015, -1.11187838],
        [-1.55895271, -1.58893375,  2.39874554,  2.15556209]]),
 Index(['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm',
        'petal_width_cm'],
       dtype='object'))

In [43]:
import numpy as np
1/np.exp(log_mod.coef_[0]), X.columns

(array([0.68995914, 0.24421361, 8.60291559, 2.59799966]),
 Index(['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm',
        'petal_width_cm'],
       dtype='object'))

In [47]:
np.round(y_probs[0], 3),  y_preds[1]

(array([0.01 , 0.676, 0.314]), 0)

With a multivariable confusion matrix, some of our labellings (True Pos., True Neg., False Pos., False Neg.) get a little warped.  We are no longer predicting one class from a null class we are classifiying into 3 distinguished classes.  

The **True** diagonal stays the same as these are properly classified observations.  


|     | Class 0 | Class 1  | Class 2 |
| --- | ------- |:--------:| -------:|
| **Pred Class 0**  | 15      | 0        | 0       |
| **Pred Class 1**    | 0       | 11       |   0     |
| **Pred Class 2**    | 0       | 1        |    11   |


It is better to stick with True and False labels with multi-class to avoid ...[_Confusion_](https://www.youtube.com/watch?v=bcYppAs6ZdI)

If you need to reffer to a False Positive or True Negative it is better to first select a specific class, such as `Class 2 ` and refer to classification or misclassification relative to said choosen class instead of the set of all classes as a whole. 

Example:
    _True Negatives relative to Class 2 are True Positives for Class 0 and Class 1._

Speaking of our Classes?  How are probabilities calculated with multi class?
- Are they Probability of `Class 0` vs. `Not Class 0`?
- Or Probability of `Class 0` vs. `Class 1` vs. `Class 2` ?

In [ ]:
# use predict_proba to find out.


Looks like our probabilities of each class all add up to 1, so it is like `Class 0` vs. `Class 1` vs. `Class 2`.

What if we wanted to create a logistic regression that has `Class 0` vs. `Class 1` & `Class 2` or just `Class 0` vs. `Class 2`?  We will cover that in a bit, but first more evaluation metrics.

---

**Classification Reports/Matrix**

Classification reports are another means of evauliation classification models and return a few metrics that are based on True Positives, False Positives and False Negatives.  

In [49]:
#  DO THIS! 
from sklearn.metrics import classification_report

print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



**Precision**  
- "How many of the items selected are relevant."
- Of the items placed into a class, how many of the are True Positives.


$$\frac{True Positives}{True Positives + False Positives}$$

**Recall**  
- "How many of the relevant items are selected."
- Of the items that were suppose to be placed into a class, how many did we accurately place.


$$\frac{True Positives}{True Positives + False Negatives}$$

**F1-Score**

F1 exists on a range of 0 - 1 where 0 is just aweful and 1 is perfection.
F1 is considered a harmonic mean as it averages Precision and Recall.  With classification models you often times have to chooise what kind of error you are willing to increase in order to reduce the other and thus you may want to optimize Precision or Recall accordingly.  If you are uncertain which you should optimize, F1 score may be the metric of choice.

$$2*\frac{precision * recall}{precision + recall}$$

**Support**
Number of true observations in given class.  The count of possible true observations.  

---

## Intro to Ensembling

Earlier we talked about building models relative to class combinations.  Distinguishing One class from all other classes or just One specific class from another specific class.  These goals are possible with Logistic Regression.

Up until this point we have used one model, but there are also Machine Learning methods that involve combining several models to come to a more refined conclusion, commonly reffered to as Ensemble Methods.

### One Vs. Rest Classification.

One vs. Rest Classification is a method that builds an individual model for each class to try to distingush said specific class from the rest of the classes.  Since we are only focusing on one class, `Class 1` these classfiers will group `Class2`, `Class3`, `Class4` into a single class of `Not Class 1`.  Same all the way through for the rest of the classes.

1 - Class1 vs. Class2, Class3, Class4  
2 - Class2 vs. Class1, Class3, Class4  
3 - Class3 vs. Class1, Class2, Class4   
4 - Class4 vs. Class1, Class2, Class3  

### One Vs. One Classification.

We train a model for every set of classes.  As more classes are added this becomes more computationally expense.  

1 - Class1 vs. Class2  
2 - Class1 vs. Class3  
3 - Class1 vs. Class4  
4 - Class2 vs. Class3  
5 - Class2 vs. Class4  
6 - Class3 vs. Class4  


#### One Vs. Rest Classifier

In [ ]:
# import ensemble method
from sklearn.multiclass import OneVsRestClassifier 
# instantiate choose model
LR = LogisticRegression()
# place the model in the ensembler
OVC = OneVsRestClassifier(LR)
# use the ensemble like a normal sklearn model.
OVC.fit(x_train, y_train)


# you can use the train test split you created earlier or do a new TTS.

In [ ]:
# use the .predict and confusion matrix the same way
y_pred = OVC.predict(x_test)

confusion_matrix(y_test, y_pred)

#### One Vs. One Classifier

In [ ]:
from sklearn.multiclass import OneVsOneClassifier
LR = LogisticRegression()
# OvO works the same as OvR

In [ ]:
# Make prediction and evaluate confusion matrix.

One Vs. One/Rest Classifiers are not restricted to fitting using Logistic Regression.  With SKLearn, any type of Classification model can be placed into the One Vs X classification ensemble.